# Simple RAG app with conversation History

In [ ]:

import os
from dotenv import load_dotenv


load_dotenv()

gemini_api_key = os.getenv("GOOGLE_API_KEY")

In [27]:

from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=gemini_api_key)


In [28]:

from langchain_core.messages import HumanMessage


llm.invoke([HumanMessage(content="What is the capital of Andhra Pradesh?")])

AIMessage(content='The capital of Andhra Pradesh is **Visakhapatnam**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ad265992-22ef-48ff-95ed-3a140c608b03-0', usage_metadata={'input_tokens': 8, 'output_tokens': 13, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}})

In [32]:

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_history = RunnableWithMessageHistory(llm, get_session_history)


In [33]:

config = { "configurable": { "session_id": "demo1" } }


In [ ]:

# Invoke with history...Providing Information
response = with_history.invoke(
    [HumanMessage(content="Hi! I am Sri, final year student at CVR COllege of Engineering.")],
    config=config
)

response.content

"Hi Sri! It's exciting to hear you're in your final year at CVR College of Engineering. What field of engineering are you studying? Are you looking for advice on job searching, further education, or anything else related to your final year and future career? Let me know how I can help!"

In [ ]:

# Asking a Question
with_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
).content


'As an AI, I don\'t know your name. You told me "Hi! I am Sri, final year student at CVR COllege of Engineering." So, I believe your name is Sri. Is that correct?'

In [41]:

# Asking a Question with a different session
config1 = { "configurable": { "session_id": "demo2" } }

with_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
).content


"I am a large language model, and I don't have access to personal information. Therefore, I don't know your name. You haven't told me!"

In [42]:

# Providing Information to a different session
with_history.invoke(
    [HumanMessage(content="My name is Riyaz.")],
    config=config1
).content

"Okay, Riyaz. It's nice to meet you! How can I help you today?"

In [43]:

with_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
).content


'Your name is Riyaz.'

In [ ]:

# Adding ChatPromptTemplate and bit complexity...
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [    ("system", "You are a friendly assistant. Answer all questions in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | llm

with_history = RunnableWithMessageHistory( chain, get_session_history, input_message_key="messages" )